In [173]:
import glob
import os
import subprocess

highlevel_dirs = sum([
    #["../tools"], 
    #sorted(glob.glob("../sem16*")),
    sorted(glob.glob("../sem18*")),
], [])

print("Highlevel dirs:", highlevel_dirs)

Highlevel dirs: ['../sem18-multiplexing']


In [174]:

def convert(n, d):
    get_ipython().system("jupyter nbconvert {} --to markdown --output {}".format(n, d))
    #subprocess.check_call(["jupyter", "nbconvert", n, "--to", "markdown", "--output", d])

for subdir in highlevel_dirs:
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
    if len(notebooks) == 1:
        convert(notebooks[0], "README")
    else:
        for n in notebooks:
            convert(n, os.path.basename(n.replace(".ipynb", "")))
        

../sem18-multiplexing ['../sem18-multiplexing/epoll-poll-select-linuxaio.ipynb', '../sem18-multiplexing/18.ipynb']
[NbConvertApp] Converting notebook ../sem18-multiplexing/epoll-poll-select-linuxaio.ipynb to markdown
[NbConvertApp] Writing 33437 bytes to ../sem18-multiplexing/epoll-poll-select-linuxaio.md
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--no-input
    Exclude input cells and output prompts from converted document. 
    This mode is ideal for generating code-free reports.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--debug
    set log level to logging.DEBUG (maximize logging output)
-y
    Answer yes to any questions instead of prompting.
--clear-output
    

In [175]:
import re

def basic_improve(fname):
    with open(fname, "r") as f:
        r = f.read()
    for b in ["\x00", "\x1B", "\x08"]:
        r = r.replace(b, "")
    with open(fname, "w") as f:
        f.write(r)
    get_ipython().system("dos2unix {}".format(fname))

def improve_md(fname):
    with open(fname, "r") as f:
        r = f.read()
    r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
    r = r.replace('\n', "SUPER_SLASH" + "_N_REPLACER")
    r = re.sub(r'\<\!--MD_BEGIN_FILTER--\>.*?\<\!--MD_END_FILTER--\>', "", r)
    r = re.sub(r'(\#SET_UP_MAGIC_BEGIN.*?\#SET_UP_MAGIC_END)', "<too much code>", r)
    r = r.replace("SUPER_SLASH" + "_N_REPLACER", '\n')
    
    def file_repl(matchobj, path=os.path.dirname(fname)):
        fname = os.path.join(path, matchobj.group(1))
        if fname.find("__FILE__") == -1:
            with open(fname, "r") as f:
                return "\n```\n" + f.read() + "\n```\n"
    
    r = r.replace("</td>", "")
    r = r.replace("</tr>", "")
    
    r = re.sub(r'\<\!--MD_FROM_FILE (.*?) --\>', file_repl, r)
    with open(fname, "w") as f:
        f.write(r)
        
def improve_file(fname):
    basic_improve(fname)
    if fname.endswith(".md"):
        improve_md(fname)


In [176]:
for sfx in [".ipynb", ".md"]:
    for hdir in highlevel_dirs:
        for fname in glob.glob("./{}/*".format(hdir) + sfx):
            improve_file(fname)

dos2unix: converting file ./../sem18-multiplexing/epoll-poll-select-linuxaio.ipynb to Unix format ...
dos2unix: converting file ./../sem18-multiplexing/18.ipynb to Unix format ...
dos2unix: Binary symbol 0x02 found at line 470
dos2unix: Skipping binary file ./../sem18-multiplexing/epoll-poll-select-linuxaio.md


In [177]:
cmds = []
add_cmd = "git add --ignore-errors "
add_cmd_f = "git add --ignore-errors -f "
for subdir in highlevel_dirs:
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        cmds.append(add_cmd + " {}/*{}".format(subdir, sfx))
    cmds.append(add_cmd_f + " -f {}/bash_popen_tmp/*.html".format(subdir))
    cmds.append(add_cmd_f + " -f {}/interactive_launcher_tmp/*.log".format(subdir))
    
def execute_cmd(cmd):
    print(">", cmd)
    get_ipython().system(cmd)
    
for cmd in cmds:
    execute_cmd(cmd)
execute_cmd("git add -u")
execute_cmd("git commit -m 'yet another update'")
execute_cmd("git push origin master")

> git add --ignore-errors  ../sem18-multiplexing/*.ipynb
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
> git add --ignore-errors  ../sem18-multiplexing/*.md
The file will have its original line endings in your working directory.
> git add --ignore-errors  ../sem18-multiplexing/*.c
fatal: pathspec '../sem18-multiplexing/*.c' did not match any files
> git add --ignore-errors  ../sem18-multiplexing/*.cpp
> git add --ignore-errors -f  -f ../sem18-multiplexing/bash_popen_tmp/*.html
fatal: pathspec '../sem18-multiplexing/bash_popen_tmp/*.html' did not match any files
> git add --ignore-errors -f  -f ../sem18-multiplexing/interactive_launcher_tmp/*.log
fatal: pathspec '../sem18-multiplexing/interactive_launcher_tmp/*.log' did not match any files
> git add -u
The file will have its original line endings in your working directory.
> git commit -m 'yet another update'
The file will have its original